In [1]:
import pudl
import pandas as pd
import numpy as np
import sqlalchemy as sa
import pudl.output.eia923 as out923
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
api_df = out923.get_fuel_cost_avg_eiaapi(out923.FUEL_COST_CATEGORIES_EIAAPI) # coal, oil, gas

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import pudl.output.eia923 as eia

In [17]:
bulk_df = eia.get_fuel_cost_avg_bulk_elec(pudl_engine=pudl_engine)

In [18]:
bulk_df.head()

,state,report_date,bulk_agg_fuel_cost_per_mmbtu,fuel_type_code_pudl
0,AK,2012-01-01,0.0,coal
1,AK,2012-02-01,0.0,coal
2,AK,2012-03-01,0.0,coal
3,AK,2012-04-01,0.0,coal
4,AK,2012-05-01,0.0,coal


In [19]:
api_df.drop(columns=['name', 'series_id', 'units']).head()

,report_date,fuel_cost_per_unit,state,fuel_type_code_pudl
0,2022-07-01,51.83,AK,coal
1,2022-06-01,59.16,AK,coal
2,2022-05-01,53.04,AK,coal
3,2022-04-01,51.69,AK,coal
4,2022-03-01,58.55,AK,coal


In [20]:
bulk_df.shape, api_df.shape

((18501, 4), (30804, 7))

In [21]:
bulk_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18501 entries, 0 to 18500
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   state                         18501 non-null  object        
 1   report_date                   18501 non-null  datetime64[ns]
 2   bulk_agg_fuel_cost_per_mmbtu  18501 non-null  float64       
 3   fuel_type_code_pudl           18501 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 578.3+ KB


In [12]:
api_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30804 entries, 0 to 10499
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   report_date          30804 non-null  datetime64[ns]
 1   fuel_cost_per_unit   18615 non-null  float64       
 2   state                30804 non-null  object        
 3   units                30804 non-null  object        
 4   series_id            30804 non-null  object        
 5   name                 30804 non-null  object        
 6   fuel_type_code_pudl  30804 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.9+ MB


In [13]:
frc_api = eia.fuel_receipts_costs_eia923(pudl_engine=pudl_engine, fill=True)

In [16]:
frc_bulk = eia.fuel_receipts_costs_eia923(pudl_engine=pudl_engine, fill=True)

In [22]:
frc_api.shape, frc_bulk.shape

((560374, 40), (560374, 37))

In [23]:
mismatch = frc_api['fuel_cost_from_eiaapi'] ^ frc_bulk['fuel_cost_from_eiaapi']

In [24]:
mismatch.agg(['sum', 'mean'])

sum     136530.000000
mean         0.243641
Name: fuel_cost_from_eiaapi, dtype: float64

In [25]:
frc_api['fuel_cost_from_eiaapi'].sum()

136530

In [26]:
frc_bulk['fuel_cost_from_eiaapi'].sum()

0